## Table of contents
* [Business Problem](#introduction)
* [Data Sources](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

**Business Problem**

One of Italian food chain (XYZ) is looking for expansion in Asian region specifically in South East Asia. Currently, management team is looking into different aspects such as how many Italian restaurants are already running & how are they performing. XYZ will consider 2 or 3 cities in phase-1 & subsequently will consider as much as in another 5 years. For initial study, XYZ is looking for geo-wise analysis in such a way that they can shortlist some of cities for budgetary approvals & start looking for infra in at-least one city. Overall, vision is to start operations in at-least 2 cities before the end of this year.  

**Data Sources**

In my analysis, I will be considering Singapore (One from top), Mumbai & Bangalore (two from mid segment), Jakarta (one from lower segment) as key locations because of major business presence & thus driving cross-continent food habits. I will consider foursquare APIs to search key venues, & specific Italian restaurant. Some of APIs to be used are as follows:

1) Locations preference for Italian food by country: https://yougov.co.uk/topics/food/articles-reports/2019/03/12/italian-cuisine-worlds-most-popular

2) Extract all key venues in a radius parameter of 6000: https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={}&v{}&limit=6000&radius=6000'

3) Extract all Italian venues: https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={}&v={}&query=Italian&limit=6000&radius=6000'

4) Extract venue rating, tips related info: https://api.foursquare.com/v2/venues/venue_id?{}?client_id={}&client_secret={}&v={}

By using above APIs, I will consider plotting each Italian resturaunt in a radius of 6000 from mid of city lat & long. In addition, other key venues will also be plotted across similar radius. Post that, I will observe possibility of entry as follows:

1) How many existing italian restraunts are there in vicinity ? 

2) How are they performing ? Using tips, rating related info 

**_Singapore Data_** 

In [29]:
#Extracting neighbourhood info using Forsquare 
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim 
import requests

In [30]:
CLIENT_ID = 'DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP' # your Foursquare ID
CLIENT_SECRET = 'F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP
CLIENT_SECRET:F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2


In [31]:
# Using city-centre lat & long from google search database
neighborhood_latitude = 1.290270 
neighborhood_longitude = 103.851959
url = 'https://api.foursquare.com/v2/venues/search?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&ll=1.290270,103.851959&v=20190404&query=restaurant&limit=6000&radius=6000'
Singapore_json = requests.get(url).json()
#Extracting nearby venues
venues = Singapore_json['response']['venues']
df = json_normalize(venues)

In [32]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]
df_filtered.head(2)

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,id
0,Jumbo Seafood Restaurant,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...","#01-01/02, Riverside Point",SG,Clarke Quay,Singapore,30 Merchant Rd,808,"[#01-01/02, Riverside Point (30 Merchant Rd), ...","[{'label': 'display', 'lat': 1.288983, 'lng': ...",1.288983,103.844812,NaN,058282,Singapore,4b62be31f964a520fc502ae3
1,Tsui Wah Restaurant,"[{'id': '58daa1558bbb0b01f18ec1d3', 'name': 'C...",#01-03 Clarke Quay,SG,Singapore,Singapore,3A River Valley Road,799,"[#01-03 Clarke Quay (3A River Valley Road), 17...","[{'label': 'display', 'lat': 1.289830768314968...",1.289831,103.844789,NaN,179020,NaN,5aae3a8ea6ec982db53fd6bf


In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]
#Extracting only non-Italian restraunt from list
df_filtered = df_filtered[df_filtered['categories'].str.contains('Italian')==False]
df_filtered.reset_index()
df_filtered.head(1)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Jumbo Seafood Restaurant,Seafood Restaurant,"#01-01/02, Riverside Point",SG,Clarke Quay,Singapore,30 Merchant Rd,808,"[#01-01/02, Riverside Point (30 Merchant Rd), ...","[{'label': 'display', 'lat': 1.288983, 'lng': ...",1.288983,103.844812,NaN,058282,Singapore,4b62be31f964a520fc502ae3


In [34]:
# Extracting Italian restraunts in the zone 
url = 'https://api.foursquare.com/v2/venues/search?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&ll=1.290270,103.851959&v=20190404&query=Italian&limit=6000&radius=6000'
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

In [35]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [36]:
#Extracting only resturants from list
df_final_s = dataframe_filtered[dataframe_filtered['categories'].str.contains('Resta')==True]
df_final_s.reset_index()
df_final_s.head(5)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Salvo Italian Restaurant & Bar,Italian Restaurant,46 Prinsep Street,SG,NaN,Singapore,NaN,1080,"[46 Prinsep Street, Singapore]","[{'label': 'display', 'lat': 1.299682426731299...",1.299682,103.849593,NaN,NaN,NaN,543bc4d1498e0fa6d2596641
1,Etna Italian Restaurant & Pizzeria,Italian Restaurant,49 Duxton Road,SG,NaN,Singapore,NaN,1643,"[49 Duxton Road, 089534, Singapore]","[{'label': 'display', 'lat': 1.278340162558995...",1.278340,103.843257,NaN,089534,NaN,4d1331fa9844b60c84d7c0d4
2,Cucina Italian Restaurant,Italian Restaurant,Aperia Mall,SG,Singapore,Singapore,12 Kallang Avenue,2523,"[Aperia Mall (12 Kallang Avenue), Singapore]","[{'label': 'display', 'lat': 1.309504786437078...",1.309505,103.863954,NaN,NaN,NaN,552a6c08498e4c2b6e6d8fa7
3,Jamie's Italian,Italian Restaurant,Forum Shopping Center,SG,Singapore,Singapore,Orchard Road,3129,"[Forum Shopping Center (Orchard Road), Singapore]","[{'label': 'display', 'lat': 1.306465055171127...",1.306465,103.828971,NaN,NaN,NaN,55b3658e498e113806297322
4,Jamie's Italian,Italian Restaurant,#01-165/166/167 VivoCity,SG,Singapore,Singapore,1 HarbourFront Walk,4552,[#01-165/166/167 VivoCity (1 HarbourFront Walk...,"[{'label': 'display', 'lat': 1.263251902076503...",1.263252,103.821257,NaN,098585,NaN,517f4c54e4b09f65c289c29c


**_Analysing Kuala Lumpur_**

In [37]:
# Using city-centre lat & long from google search database
url = 'https://api.foursquare.com/v2/venues/search?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&ll=3.1390,101.6869&v=20190404&query=restaurant&limit=6000&radius=6000'
kl_json = requests.get(url).json()
#Extracting nearby venues
venues_j = kl_json['response']['venues']
df2 = json_normalize(venues_j)
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
df_filtered_j = df2.loc[:, filtered_columns]
df_filtered_j.head(2)

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,id
0,IKEA Restaurant & Cafe,"[{'id': '4bf58dd8d48988d1c6941735', 'name': 'S...","IKEA (Level 2), Jalan Cochrane",MY,Kuala Lumpur,Malaysia,NaN,3916,"[IKEA (Level 2), Jalan Cochrane, 55100 Cheras,...","[{'label': 'display', 'lat': 3.135085241543385...",3.135085,101.721917,Taman Maluri,55100,Kuala Lumpur,5646b5d6498e6d752c6f5bb4
1,Restaurant Sin Kee (新记(中西)餐室),"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...","No. 194, Jalan Tun Sambathan",MY,Kuala Lumpur,Malaysia,Brickfields,890,"[No. 194, Jalan Tun Sambathan (Brickfields), 5...","[{'label': 'display', 'lat': 3.131061, 'lng': ...",3.131061,101.685905,NaN,50470,Kuala Lumpur,4d9c5d7ba1ec8cfaf87ae849


In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered_j['categories'] = df_filtered_j.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered_j.columns = [column.split('.')[-1] for column in df_filtered_j.columns]
df_filtered_j.head(2)
#Extracting only non-Italian restraunt from list
df_filtered_j = df_filtered_j[df_filtered_j['categories'].str.contains('Italian')==False]
df_filtered_j.reset_index()
df_filtered_j.head(1)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,IKEA Restaurant & Cafe,Scandinavian Restaurant,"IKEA (Level 2), Jalan Cochrane",MY,Kuala Lumpur,Malaysia,NaN,3916,"[IKEA (Level 2), Jalan Cochrane, 55100 Cheras,...","[{'label': 'display', 'lat': 3.135085241543385...",3.135085,101.721917,Taman Maluri,55100,Kuala Lumpur,5646b5d6498e6d752c6f5bb4


In [39]:
# Extracting italian restraunts in the zone 
url = 'https://api.foursquare.com/v2/venues/search?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&ll=3.1390,101.6869&v=20190404&query=Italian&limit=6000&radius=6000'
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues_j = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues_j)

In [40]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [41]:
#Extracting only restraunt from list
df_final_j = dataframe_filtered[dataframe_filtered['categories'].str.contains('Resta')==True]
df_final_j.reset_index()
df_final_j.head(2)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Svago Cucina e Musica & Limoncello Italian Bis...,Italian Restaurant,Suria KLCC,MY,Kuala Lumpur,Malaysia,G45B/136,3472,"[Suria KLCC (G45B/136), 50088 Kuala Lumpur, Ku...","[{'label': 'display', 'lat': 3.156478937470702...",3.156479,101.712774,NaN,50088,Kuala Lumpur,5100c8e8e4b0540093ab7699
1,Aria Traditional Italian Restaurant,Italian Restaurant,Plaza Damansara,MY,Kuala Lumpur,Malaysia,44G,3856,"[Plaza Damansara (44G), 50490 Kuala Lumpur, Ku...","[{'label': 'display', 'lat': 3.149221929329950...",3.149222,101.653747,NaN,50490,Kuala Lumpur,4cd9018ca99d3704039f08ce


**_Analysing Mumbai_**

In [55]:
# Using city-centre lat & long from google search database
url = 'https://api.foursquare.com/v2/venues/search?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&ll=19.0760,72.8777&v=20190404&query=restaurant&limit=12000&radius=12000'
m_json = requests.get(url).json()
#Extracting nearby venues
venues_m = m_json['response']['venues']
df2 = json_normalize(venues_m)
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
df_filtered_m = df2.loc[:, filtered_columns]
df_filtered_m.head(1)

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,id
0,Subway restaurant,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...","Ostwal orbit,shop no 2\3,Kanakia junction",IN,Mumbai,India,Mira bhayandar road,4,"[Ostwal orbit,shop no 2\3,Kanakia junction (Mi...","[{'label': 'display', 'lat': 19.0759837, 'lng'...",19.075984,72.877656,NaN,Mahārāshtra,4ffff479e4b0da072a4c510e


In [43]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered_m['categories'] = df_filtered_m.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered_m.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
df_filtered_m.head(1)
#Extracting only non-Italian restraunt from list
df_filtered_m = df_filtered_m
df_filtered_m.reset_index()
df_filtered_m.head(1)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Subway restaurant,Sandwich Place,"Ostwal orbit,shop no 2\3,Kanakia junction",IN,Mumbai,India,Mira bhayandar road,4,"[Ostwal orbit,shop no 2\3,Kanakia junction (Mi...","[{'label': 'display', 'lat': 19.0759837, 'lng'...",19.075984,72.877656,NaN,NaN,Mahārāshtra,4ffff479e4b0da072a4c510e


In [44]:
# Extracting Italian restraunts in the zone 
url = 'https://api.foursquare.com/v2/venues/search?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&ll=19.0760,72.8777&v=20190404&query=pizza&limit=12000&radius=12000'
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues_m = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues_m)

In [45]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [46]:
#Extracting only restraunt from list
df_final_m = dataframe_filtered
df_final_m.reset_index()
df_final_m.head(1)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Pizza Hut,Pizza Place,Oberoi Mall,IN,Mumbai,India,Goregaon,4,"[Oberoi Mall (Goregaon), Mumbai, Mahārāshtra, ...","[{'label': 'display', 'lat': 19.0759837, 'lng'...",19.075984,72.877656,NaN,NaN,Mahārāshtra,4ff2ae9ee4b0365ccbfbb6cb


**_Analysing Bangalore_**

In [47]:
# Using city-centre lat & long from google search database
url = 'https://api.foursquare.com/v2/venues/search?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&ll=12.9716,77.5946&v=20190404&query=restaurant&limit=12000&radius=12000'
b_json = requests.get(url).json()
#Extracting nearby venues
venues_b = b_json['response']['venues']
df3 = json_normalize(venues_b)
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
df_filtered_b = df3.loc[:, filtered_columns]
df_filtered_b.head(2)

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,id
0,Ujwal Bar & Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...","402-A, Millers Road, Vasanthnagar,",IN,Bangalore,India,NaN,2302,"[402-A, Millers Road, Vasanthnagar,, Bangalore...","[{'label': 'display', 'lat': 12.99227960861634...",12.99228,77.594473,NaN,560052,Karnātaka,4b6ee375f964a52029cf2ce3
1,Mayuri Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",J C Road,IN,Bangalore,India,NaN,2279,"[J C Road, Bangalore 560002, Karnātaka, India]","[{'label': 'display', 'lat': 12.95612026676884...",12.95612,77.580845,NaN,560002,Karnātaka,4d01ec0e3be26dcb537475e4


In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered_b['categories'] = df_filtered_b.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered_b.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
df_filtered_b.head(1)
#Extracting only non-Italian restraunt from list
df_filtered_b = df_filtered_b[df_filtered_b['categories'].str.contains('pizza')==False]
df_filtered_b.reset_index()
df_filtered_b.head(1)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Ujwal Bar & Restaurant,Indian Restaurant,"402-A, Millers Road, Vasanthnagar,",IN,Bangalore,India,NaN,2302,"[402-A, Millers Road, Vasanthnagar,, Bangalore...","[{'label': 'display', 'lat': 12.99227960861634...",12.99228,77.594473,NaN,560052,Karnātaka,4b6ee375f964a52029cf2ce3


In [49]:
# Extracting Italian restraunts in the zone 
url = 'https://api.foursquare.com/v2/venues/search?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&ll=12.9716,77.5946&v=20190404&query=pizza&limit=12000&radius=12000'
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues_b = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues_b)

In [50]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [56]:
#Extracting only restraunt from list
df_final_b = dataframe_filtered
df_final_b.reset_index()
df_final_b.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Domino's Pizza,Pizza Place,"30, Rest House Rd",IN,Bangalore,India,NaN,1364,"[30, Rest House Rd, Bangalore 560001, Karnātak...","[{'label': 'display', 'lat': 12.97368493806726...",12.973685,77.606991,560001,Karnātaka,4c092177ffb8c9b699526961
1,Pizza Corner India Pvt Ltd,Pizza Place,"Unit No FCO63rd Floor, Food CourtMantri Mall, ...",IN,Bangalore,India,NaN,1082,"[Unit No FCO63rd Floor, Food CourtMantri Mall,...","[{'label': 'display', 'lat': 12.981316, 'lng':...",12.981316,77.595030,560003,Karnātaka,523da0860493a03a0a7bbfaf
2,Pizza Stop,Pizza Place,NaN,IN,Bangalore,India,Cunningham Road,1649,"[Cunningham Road, Bangalore 560052, Karnātaka,...","[{'label': 'display', 'lat': 12.98642070767535...",12.986421,77.594809,560052,Karnātaka,4f05b75c4901346082afbc46
3,Domino's Pizza,Pizza Place,Sai Baba Colony,IN,Bangalore,India,NaN,62,"[Sai Baba Colony, Bangalore 560037, Karnātaka,...","[{'label': 'display', 'lat': 12.972, 'lng': 77...",12.972000,77.595000,560037,Karnātaka,4ce90bc5595cb1f78384cc14
4,Domino's Pizza,Pizza Place,NaN,IN,NaN,India,NaN,1365,[India],"[{'label': 'display', 'lat': 12.97349969813273...",12.973500,77.607034,NaN,NaN,56a37e72498ed53785efe2a6
5,Pizza stop,Pizza Place,NaN,IN,Bangalore,India,NaN,608,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.97298414297131...",12.972984,77.600028,NaN,Karnātaka,4e0dbfa1483bc2b5f47a3a1c
6,Pizza Hut,Pizza Place,"66/1, Coles Rd",IN,Bangalore,India,Frazer Town,3387,"[66/1, Coles Rd (Frazer Town), Bangalore 56000...","[{'label': 'display', 'lat': 12.99561827424566...",12.995618,77.613775,560005,Karnātaka,4c03eaeaf56c2d7f6f7e1d66
7,Domino's Pizza,Pizza Place,"Trinity Metro Station\nNAMMA METRO, Station C...",IN,Bangalore,India,NaN,2434,"[Trinity Metro Station, NAMMA METRO, Station ...","[{'label': 'display', 'lat': 12.973, 'lng': 77...",12.973000,77.617000,560001,Karnātaka,52d51213498e01123ebcb0da
8,Pizza Hut,American Restaurant,"3rd Flr, Mantri Square Mall, Sampige Rd",IN,Bangalore,India,Malleshwaram,3383,"[3rd Flr, Mantri Square Mall, Sampige Rd (Mall...","[{'label': 'display', 'lat': 12.99149480853619...",12.991495,77.571023,560066,Karnātaka,4ff941eee4b0cc077ed9c6a0
9,Pizza Hut,Pizza Place,"185, Dheena Complex",IN,Bangalore,India,Brigade Rd,1398,"[185, Dheena Complex (Brigade Rd), Bangalore 5...","[{'label': 'display', 'lat': 12.97317888793035...",12.973179,77.607392,560001,Karnātaka,4bc06339f8219c74087db110


In this section, we have carved out restaraunt or specifically Italian only. This will act as a key pillar for methodology & final strategy to arrive on right place for investment.    

**Methodology**

We can now see the density of Italian & other restaurants for each city. We will utilize below steps to ensure that there is market for Italian food & location is good to open new restaurant. 

1) Density of restaurants & Italian restaurants nearby

2) Higher density means there is good footfall (higher spending power/demand)

3) Higher restaurant density but a few Italian is good pick

Further, ratings/tips can be used to see if existing restaurants are performing well or not. In case lower rating, it could be good pick if no restaurants in vicinity. 

**Analysis**

_Singapore_

In [110]:
# Plot for Singapore with Italian & other resturants
import folium
map = folium.Map(location=[1.29, 103.85], zoom_start=13.5)
for i in range(0,len(df_filtered)):
    folium.Circle([df_filtered.iloc[i]['lat'], df_filtered.iloc[i]['lng']],color='blue',fill=True,fill_color='blue',fill_opacity=0.7).add_to(map)
for lat, lng, label in zip(df_final_s.lat, df_final_s.lng, df_final_s.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=2,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map)
map

As can be seen above, there is good density of Italian restaurants across whole Singapore, however, there seems a patch of area near **China town** & **Clarke quay**, without any Italian restaurants. Similarly, it can be seen for **City hall**, but not as open as other two. With this, Singapore can be chosen as one of location, however, lets explore others - may be more open.     

_Jakarta_

In [118]:
# Plot for Jakarta with Italian & other resturants
import folium
map = folium.Map(location=[3.1390,101.6869], zoom_start=13)
for i in range(0,len(df_filtered_j)):
    folium.Circle([df_filtered_j.iloc[i]['lat'], df_filtered_j.iloc[i]['lng']],color='blue',fill=True,fill_color='blue',fill_opacity=0.7).add_to(map)
for lat, lng, label in zip(df_final_j.lat, df_final_j.lng, df_final.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=2,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map)
map

Same as Singapore, Jakarta also have plenty of Italian food restaurants. It can also be seen that areas such as **Brickfields** have just one Italian food restaurants and many for other cuisines. It can be one of location for new Italian food restaurant.  

_Mumbai_

In [176]:
# Plot for Mumbai with Italian & other resturants
import folium
map = folium.Map(location=[19.0760,72.8777], zoom_start=12)
for i in range(0,len(df_filtered_m)):
    folium.Circle([df_filtered_m.iloc[i]['lat'], df_filtered_m.iloc[i]['lng']],color='blue',fill=True,fill_color='blue',fill_opacity=0.7).add_to(map)
for lat, lng, label in zip(df_final_m.lat, df_final_m.lng, df_final.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=2,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map)
map

Considering the density above, it seems all places have equal or good mix of Italian food restaurants (**Bandra**, **Chembur**). However, they may be catering to relevant segment - premium, mid or lower. Still, there will be risk & require further study into type of customers visiting into & average ticket size.       

_Bangalore_

In [183]:
import folium
map = folium.Map(location=[12.9716,77.5946], zoom_start=12)
for i in range(0,len(df_filtered_b)):
    folium.Circle([df_filtered_b.iloc[i]['lat'], df_filtered_b.iloc[i]['lng']],color='blue',fill=True,fill_color='blue',fill_opacity=0.7).add_to(map)
for lat, lng, label in zip(df_final_b.lat, df_final_b.lng, df_final.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=2,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map)
map

Similar to Mumbai, there seems good mix of restaurants serving Italian cuisine. There are some empty patches (**Shivajinagar**,**Shantinagar**,**Jayanagar**), but still they seem crowded. There is huge demand for Italian food for sure.      

**Results & Discussion**

As we could see that based on existing density, clearly, Singapore & Kuala Lumpur have clear patches to start with phase 1. However, It could be due to lower density or demand, specifically in Kuala Lumpur. At the same time, Mumbai & Bangalore have reasonable density, therefore, possibility to gain some market share in shorter time frame but with affordable price range or looking more deeply to how existing food-chains are performing.
In order to avoid the risk, consider Singapore & Bangalore as key locations for phase 1. First, open patches of Singapore will provide good place for growth & capture market share. Second, Bangalore have demand for Italian food, however, need to be specific in terms of price segment, as higher competition may not provide good grounds for healthy margin or even sales in first place.

Now, lets analyze the some of restaurants for tips/ratings for Singapore & Bangalore.

In [58]:
# Italian restaurant rating in Singaplore 
url1 = 'https://api.foursquare.com/v2/venues/5993ed99356b497a3d3df414?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&v=20190404'
result1 = requests.get(url1).json()
try:
    print(result1['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.2


In [57]:
# Italian restaurant rating in Bangalore 
url1 = 'https://api.foursquare.com/v2/venues/4bc06339f8219c74087db110?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&v=20190404'
result1 = requests.get(url1).json()
try:
    print(result1['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.0


Not all restaurants are rated, but some of them have more than average rating. It means there is acceptance or liking for Italian food & therefore, we may consider entering mentioned cities in initial phase.

**Conclusion & Future Direction**

Based on analysis above, it can be seen that all cities have good acceptance, however, I have recommended **Singapore** & **Bangalore** as first entry points in Asia. In order to be more specific in 2nd phase, additional information on demographics, & income may be crucial. Tips related information will be good to include, but could not use the same due to free API restrictions by foursquare. Other two cities, Mumbai & Kuala Lumpur also have good potential, so shall be explored more in phase 2.      